In [53]:
import numpy as np
from utils import UtilsRender
from game_entities import Arrow

# %%
from importlib import reload
import utils
import collision_resolver
utils = reload(utils)
ur = utils.UtilsRender()
collision_resolver = reload(collision_resolver)
CollisionResolver = collision_resolver.CollisionResolver

# Initial setup

In [2]:
w = 1920
h = 1080
radii = 25
X = np.array(
    [[  1275,  530],
    [1307,  470],
    [1310,  700],
    [1390,  560],
    [100,  100],
    ], dtype=np.float32
)
R = np.array([radii, radii, radii, radii, radii], dtype=np.float32)
M = np.array([1, 1, 1, 1, 1], dtype=np.float32)


In [5]:
# ur.render_snapshot(X, R)

# Collision Resolver

In [6]:
V = np.array(
    [[4, 4],
    [2, -2],
    [-1.5, -1.5],
    [3, 3],
    [-3, 0],
    ], dtype=np.float32
)
cr = CollisionResolver(R=R, M=M, boundaries=(w, h))
# ur.render_snapshot(X_next, R)

### Video of caps moving and colliding

In [7]:
def get_cap_velocity_modulus(V):
        """
        Get the velocity modulus of the cap
        Returns a matrix of shape (n, 1)
        """
        return np.linalg.norm(V, axis=1)

def is_system_moving(V):
    """
    Check if norm of V matrix is >= min_velocity
    """
    min_velocity = 0.1
    caps_mod_velocity = get_cap_velocity_modulus(V)
    return np.all(caps_mod_velocity < min_velocity)

In [8]:
V

array([[ 4. ,  4. ],
       [ 2. , -2. ],
       [-1.5, -1.5],
       [ 3. ,  3. ],
       [-3. ,  0. ]], dtype=float32)

In [9]:
%%time
positions = [X]
X_i = X.copy()

for i in range(5000):
    V_next = cr.resolve_collision(X=X_i, V=V)
    X_next = X_i + V_next
    positions.append(X_next)
    X_i = X_next
    V = V_next
    if is_system_moving(V_next):
        print(f"System stopped at iteration {i}")
        break

CPU times: user 125 ms, sys: 0 ns, total: 125 ms
Wall time: 124 ms


In [10]:
X_i

array([[1083.5006  ,  948.73486 ],
       [1574.2303  ,  714.5263  ],
       [ 470.23416 ,  334.85095 ],
       [ 686.77936 ,  772.81165 ],
       [  37.580322,  943.47705 ]], dtype=float32)

In [11]:
V_next

array([[ 5.0631113 ,  3.7247574 ],
       [-1.675611  , -3.074023  ],
       [ 0.54850006,  1.4674139 ],
       [ 1.9307957 ,  2.42199   ],
       [ 2.7615526 ,  0.24380466]], dtype=float32)

In [47]:
#ur.render_motion(positions=positions, radius=R, add_delay=20)

In [12]:
# ur.render_snapshot(positions[-1], R)

In [13]:
V_next = cr.resolve_collision(X=X_i, V=V)

In [14]:
V_next

array([[ 5.0631113 ,  3.7247574 ],
       [-1.675611  , -3.074023  ],
       [ 0.54850006,  1.4674139 ],
       [ 1.9307957 ,  2.42199   ],
       [ 2.7615526 ,  0.24380466]], dtype=float32)

In [31]:
def apply_smooth_friction(V, velocity_threshold=0.25):
    scale_1 = 1.1
    scale_2 = 1.3
    scale_3 = 1.2
    scale_4 = 1.5

    # Compute the speed (modulus) for each cap
    speeds = np.linalg.norm(V, axis=1)
    
    # Create masks for each scale based on the speed
    mask_1 = speeds > 5
    mask_2 = np.logical_and(speeds > 3, speeds <= 5)
    mask_3 = np.logical_and(speeds > 2, speeds <= 3)
    mask_4 = np.logical_and(speeds > velocity_threshold, speeds <= 2)
    mask_stop = speeds <= velocity_threshold

    # Initialize new velocity matrix
    V_new = V.copy()

    # Apply scales to speeds and update velocities accordingly
    speeds_new = speeds.copy()
    speeds_new[mask_1] /= scale_1
    speeds_new[mask_2] /= scale_2
    speeds_new[mask_3] /= scale_3
    speeds_new[mask_4] /= scale_4
    speeds_new[mask_stop] = 0

    # Update V_new to reflect the new speeds while maintaining direction
    for i in range(len(V)):
        if speeds[i] > 0:  # Avoid division by zero
            V_new[i] = V[i] / speeds[i] * speeds_new[i]

    return V_new

In [160]:
V = np.array(
    [[4, 4],
    [2, -2],
    [-1.5, -1.5],
    [3, 3],
    [-3, 0.05],
    ], dtype=np.float32
) * 2
# Apply smooth friction
V_new = apply_smooth_friction(V)

In [161]:
V

array([[ 8. ,  8. ],
       [ 4. , -4. ],
       [-3. , -3. ],
       [ 6. ,  6. ],
       [-6. ,  0.1]], dtype=float32)

In [162]:
V_new

array([[ 7.2727265 ,  7.2727265 ],
       [ 3.6363633 , -3.6363633 ],
       [-2.3076925 , -2.3076925 ],
       [ 5.4545455 ,  5.4545455 ],
       [-5.4545455 ,  0.09090909]], dtype=float32)

### Run Motion

In [176]:
V = np.array(
    [[1, 1.5],
    [2, -2],
    [-1.5, -1.5],
    [2.4, -2.1],
    [-3, 0],
    ], dtype=np.float32
) * 2
cr = CollisionResolver(R=R, M=M, boundaries=(w, h))

In [177]:
cr = CollisionResolver(R=R, M=M, boundaries=(w, h))
positions = [X]
X_i = X.copy()

for i in range(5000):
    V_next = cr.resolve_collision(X=X_i, V=V)
    if i % 50 == 0:
        V_next = apply_smooth_friction(V=V_next)
    X_next = X_i + V_next
    positions.append(X_next)
    X_i = X_next
    V = V_next
    if is_system_moving(V_next):
        print(f"System stopped at iteration {i}")
        break

System stopped at iteration 600


In [178]:
ur.render_motion(positions=positions, radius=R, add_delay=5)

: 

# Run Race End to End horizontally

In [50]:
w = 1920
h = 1080
radii = 25
X = np.array(
    [[radii + 5, 50],
     [radii + 5, 150],
     [radii + 5, 250],
     [radii + 5, 350],
     [radii + 5, 450],
     [radii + 5, 550],
     [radii + 5, 650],
     [radii + 5, 750],
     [radii + 5, 850],
     [radii + 5, 950]], dtype=np.float32
)

R = np.array([radii] * 10, dtype=np.float32)
M = np.array([1] * 10, dtype=np.float32)

V = np.array(
    [[1, 0],
     [2, 0],
     [3, 0],
     [4, 0],
     [5, 0],
     [6, 0],
     [7, 0],
     [8, 0],
     [9, 0],
     [10, 0]], dtype=np.float32
)

In [51]:
cr = CollisionResolver(R=R, M=M, boundaries=(w, h))
positions = [X]
X_i = X.copy()

for i in range(5000):
    V_next = cr.resolve_collision(X=X_i, V=V)
    if i % 50 == 0:
        V_next = apply_smooth_friction(V=V_next)
    X_next = X_i + V_next
    positions.append(X_next)
    X_i = X_next
    V = V_next
    if is_system_moving(V_next):
        print(f"System stopped at iteration {i}")
        break

System stopped at iteration 900


In [ ]:
ur.render_motion(positions=positions, radius=R, add_delay=5)